In [17]:
def compute_gradient(y: torch.Tensor, x: torch.Tensor, grad_outputs=None):
    """computes dy/dx

    Args:
        y (torch.Tensor): functional
        x (torch.Tensor): positions
        grad_outputs (torch.Tensor): stuff to write output to.

    Returns:
        grad (torch.Tensor): gradient of y wrt x
    """
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(
        y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad


In [37]:
import torch

def compute_radius_of_curvature(y: torch.Tensor, x: torch.Tensor):
    """Compute the radius of curvature of a function y(x).

    Args:
        y (torch.Tensor): The functional representing the curve.
        x (torch.Tensor): The positions.

    Returns:
        radius (torch.Tensor): The radius of curvature.
    """
    # Compute the first derivative of y with respect to x
    dy_dx = compute_gradient(y, x)

    # Compute the second derivative of y with respect to x
    d2y_dx2 = compute_gradient(dy_dx, x)
    dist1 = torch.einsum(
                '...n,...n->...',(dy_dx.detach()), d2y_dx2.detach())
    dist=torch.einsum('i,i->', dy_dx, d2y_dx2)
    
    # Calculate the radius of curvature using the formula:
    # R = ((1 + (dy/dx)^2)^(3/2)) / |d^2y/dx^2|
    radius = (torch.abs(( dy_dx**2)**3)) / torch.sqrt(((torch.abs(d2y_dx2)**2)*(torch.abs(dy_dx))**2)-(dist))

    return radius

# Define a function y(x) as torch tensors
x = torch.tensor([11.0, 2.0, 3.0, 4.0, 5.0], requires_grad=True)
y = x ** 2  # Example quadratic function

# Calculate the radius of curvature
radius = compute_radius_of_curvature(y, x)
dy_dx = compute_gradient(y, x)
print(dy_dx)

# Print the radius of curvature
print("Radius of Curvature:", radius)


tensor([22.,  4.,  6.,  8., 10.], grad_fn=<MulBackward0>)
Radius of Curvature: tensor([2646060.2500,          nan,    7033.6562,   20988.3184,   57735.0234],
       grad_fn=<DivBackward0>)


In [39]:
x[1]

tensor(2., grad_fn=<SelectBackward0>)

In [30]:
import torch
import torch.nn.functional as F
import numpy as np

# Create two tensors of the same size (you would typically use real data)
inter_grad = torch.tensor([[1.0, 2.0, 3.0],
                          [4.0, 5.0, 6.0]])
d_pos = torch.tensor([[0.1, 0.2, 0.3],
                      [0.4, 0.5, 0.6]])

# Normalize the tensors along dim=-1
normalized_inter_grad = F.normalize(inter_grad, dim=-1)
normalized_d_pos = F.normalize(d_pos, dim=-1)

# Compute the dot product and take the absolute value and square root
dist = torch.einsum(
                '...n,...n->...',(inter_grad.detach()), d_pos.detach())

print("Distance:")
print("Distance:",np.shape(dist))
print(dist.unsqueeze(-1))


Distance:
Distance: torch.Size([2])
tensor([[1.4000],
        [7.7000]])
